In [1]:
import datetime

In [2]:
today = datetime.datetime.today()

In [3]:
residents = [
    ("senior51", 5),
    ("senior52", 5),
    ("senior41", 4),
    ("senior42", 4),
    ("junior31", 3),
    ("junior32", 3),
    ("junior21", 2),
    ("junior22", 2)
]

In [4]:
seniors = [x for x in residents if x[1] in (4,5)]
juniors = [x for x in residents if x[1] in (2,3)]

In [5]:
def rand_from_l(l):
    index = int(random.random() * len(l))
    return l[index]

In [6]:
def is_fri_sat_sun(day):
    return day.weekday() in (4,5,6)
def is_fri_sun(day):
    return day.weekday() in (4,6)

def mon_thru_fri(day):
    return day.weekday() in (0,1,2,3,4)

In [7]:
today = datetime.datetime(year=2019, month=7, day=1)

In [8]:
import random

holidays = {
    "thanks": {
        "days": [datetime.datetime(year=2019, month=11, day=28) + datetime.timedelta(days=x) for x in range(4)],
        "senior": [('senior41', 4), ('senior41', 4), ('senior41', 4), ('senior41', 4)],
        "junior": [('junior21', 2), ('junior21', 2), ('junior22', 2), ('junior22', 2)]
    },
    #"july4": None,
    #"memorial": None,
    #"labor": None
}
thanksgiving = []

def is_holiday(day):
    for k,v in holidays.items():
        if day in v['days']:
            my_index = v['days'].index(day)
            return v['senior'][my_index], v['junior'][my_index]
    return None

def random_call_schedule():
    today = datetime.datetime(2019, 7, 1, 0, 0)
    weekend_senior = seniors[0]
    fri_sun_junior = None
    calls = []
    for i in range(182):
        call_senior = rand_from_l(seniors)
        call_junior = rand_from_l(juniors)
        day = today + datetime.timedelta(days=i)
        retval = is_holiday(day)
        if retval is not None:
            call_senior, call_junior = retval
            calls.append([day, call_senior, call_junior])
            continue
        
        if mon_thru_fri(day):
            if len(calls) != 0:
                last_senior = calls[-1][1]
                last_junior = calls[-1][2]
                while call_junior == last_junior:
                    call_junior = rand_from_l(juniors)
                while call_senior == last_senior:
                    call_senior = rand_from_l(seniors)
        
        if day.weekday() == 4:
            weekend_senior = call_senior
            fri_sun_junior = call_junior
            
        if is_fri_sat_sun(day):
            call_senior = weekend_senior
        if is_fri_sun(day):
            call_junior = fri_sun_junior
        while day.weekday() == 5 and call_junior == fri_sun_junior:
            call_junior = rand_from_l(juniors)

        calls.append([day, call_senior, call_junior])
    return calls

In [9]:
def count_totals(calls):
    names = {}
    for elem in calls:
        senior = elem[1][0]
        junior = elem[2][0]
        if senior not in names:
            names[senior] = 0
        if junior not in names:
            names[junior] = 0
        names[senior] += 1
        names[junior] += 1
    return names

def count_weekends(calls):
    names = {}
    for elem in calls:
        day_of_week = elem[0].weekday()
        if day_of_week not in (5,6):
            continue
        senior = elem[1][0]
        junior = elem[2][0]
        if senior not in names:
            names[senior] = 0
        if junior not in names:
            names[junior] = 0
        names[senior] += 1
        names[junior] += 1
    return names

In [10]:
def score_it(calls):
    totals = count_totals(calls)
    values = list(totals.values())
    p1 = max(values) - min(values)
    
    totals = count_weekends(calls)
    values = list(totals.values())
    p2 = max(values) - min(values)
    return p1 + p2

In [13]:
def get_best_schedule(n_bootstraps=1000):
    best_call = random_call_schedule()
    for i in range(n_bootstraps):
        my_call = random_call_schedule()
        my_score = score_it(my_call)
        if score_it(my_call) < score_it(best_call):
            best_call = my_call
    return best_call

In [14]:
import time
start_time = datetime.datetime.now()
best_call = get_best_schedule(100000)
end_time = datetime.datetime.now()
print(end_time - start_time)

0:01:15.817208


In [15]:
dows = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}
def print_call(calls, person=None):
    if person is None:
        people = [x[0] for x in residents]
    else:
        people = [person]
    for c in calls:
        if c[1][0] in people or c[2][0] in people:
            dow = dows[c[0].weekday()]
            print([dow] + [str(x) for x in c])

In [22]:
def fixed_length(s):
    return s[-2:]

def print_call_grid(calls):
    s = "(NA,NA)"
    for c in calls:
        dow = dows[c[0].weekday()]
        if dow == "Sunday":
            joiner = "\n"
        else:
            joiner = ","
        day_s = "(%s:%s)" % (fixed_length(c[1][0]), fixed_length(c[2][0]))
        s = "%s%s%s" % (s, joiner, day_s)
    print(s)

In [23]:
print_call_grid(best_call)

(NA,NA),(41:22),(42:21),(41:22),(42:21),(41:31),(41:32)
(41:31),(42:32),(41:22),(52:32),(51:31),(42:21),(42:32)
(42:21),(41:32),(51:22),(41:21),(52:32),(41:31),(41:21)
(41:31),(51:21),(42:22),(41:21),(51:31),(52:22),(52:21)
(52:22),(41:32),(51:31),(41:22),(51:31),(52:22),(52:32)
(52:22),(51:21),(52:32),(51:22),(41:31),(42:32),(42:22)
(42:32),(52:22),(42:21),(52:31),(41:32),(42:21),(42:22)
(42:21),(52:22),(42:21),(41:22),(52:32),(51:22),(51:31)
(51:22),(52:32),(42:22),(52:31),(51:32),(42:31),(42:21)
(42:31),(52:21),(51:31),(52:22),(42:31),(41:32),(41:22)
(41:32),(52:31),(42:22),(51:31),(42:21),(51:32),(51:21)
(51:32),(42:31),(52:21),(42:31),(51:22),(42:32),(42:31)
(42:32),(52:31),(51:21),(42:22),(51:21),(52:22),(52:21)
(52:22),(41:32),(42:31),(41:22),(42:31),(51:32),(51:21)
(51:32),(52:21),(42:31),(51:32),(42:22),(41:31),(41:32)
(41:31),(51:32),(42:22),(52:31),(42:22),(51:32),(51:21)
(51:32),(41:22),(51:31),(41:21),(51:22),(41:21),(41:31)
(41:21),(42:31),(41:21),(42:32),(51:31),(41:32),

In [41]:
"""
Only the first two headers in this list are required, the rest are optional.

Important: The headers must be in English as shown below. If any event details have commas (like the location example below), you can include them by using quotation marks around the text

Subject
The name of the event, required.
Example: Final exam

Start Date
The first day of the event, required.
Example: 05/30/2020

Start Time
The time the event begins.
Example: 10:00 AM

End Date
The last day of the event.
Example: 05/30/2020

End Time
The time the event ends.
Example: 1:00 PM
All Day Event
Whether the event is an all-day event. Enter True if it is an all-day event, and False if it isn't.
Example: False
Description
Description or notes about the event.
Example: 50 multiple choice questions and two essay questions 
Location
The location for the event.
Example: "Columbia, Schermerhorn 614"
Private
Whether the event should be marked private. Enter True if the event is private, and False if it isn't.
Example: True
The examples above would create the event "Final exam" on May 30, 2020 from 10:00 AM to 1:00 PM. The location would be "Columbia, Schermerhorn 614," the description would be "50 multiple choice questions and two essay questions," and it would be a private event.
"""
def date_to_s(d):
    return "%s/%s/%s" % (d.month, d.day, d.year)


def to_gcal(calls, fname):
    table = []
    
    for c in calls:
        dow = dows[c[0].weekday()]
        my_day = date_to_s(c[0])
        subject = c[1][0]
        table.append([c[1][0], my_day])
        table.append([c[2][0], my_day])
    df = pd.DataFrame(table, columns=["Subject","Start Date"])
    df.to_csv(fname, index=None)
    return df

In [42]:
df = to_gcal(best_call, "call.csv")

In [28]:
import pandas as pd

In [43]:
count_totals(best_call)

{'senior41': 45,
 'junior22': 46,
 'senior42': 46,
 'junior21': 47,
 'junior31': 45,
 'junior32': 44,
 'senior52': 45,
 'senior51': 46}

In [44]:
count_weekends(best_call)

{'senior41': 14,
 'junior32': 13,
 'junior31': 13,
 'senior42': 12,
 'junior21': 13,
 'senior52': 14,
 'junior22': 13,
 'senior51': 12}